In [1]:
import gc
import concurrent.futures

from KeywordSearch import loader, indexing, utils, kwsearch

Please ignore the syntax warnings as small integers in CPython are singletons
Using `is` instead of `=` for comparison in performance-critical code is acceptable


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\10022\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\10022\Documents\GitHub\ttds-group-project\KeywordSearch\indexing.py:105: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if completed_jobs % 5 is 0: gc.collect()
c:\Users\10022\Documents\GitHub\ttds-group-project\KeywordSearch\utils.py:252: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (complete_counter % 5 is 0):


In [2]:
import requests
link = "https://www.gutenberg.org/browse/recent/last30"

In [3]:
response = requests.get(link)
response.status_code == 200
x = requests.get("https://www.gutenberg.org/ebooks/72911.txt.utf-8")

In [4]:
x.content.decode(encoding="utf-8", errors="ignore")

"\ufeffThe Project Gutenberg eBook of The botanist's repository for new and rare plants; vols 3 & 4 [of 10]\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\r\n\r\nTitle: The botanist's repository for new and rare plants; vols 3 & 4 [of 10]\r\n\r\n\r\nAuthor: active 1799-1828 Henry Cranke Andrews\r\n\r\nRelease date: February 9, 2024 [eBook #72911]\r\n\r\nLanguage: English\r\n\r\nOriginal publication: London: The author, 1797\r\n\r\nCredits: Chuck Greif and the Online Distributed Proofreading Team at https://www.pgdp.net (This file was produced from images generously mad

In [5]:
with open("tmp.html", 'w') as f:
    f.write(response.text)

In [6]:
index = loader.load_merged_index(max_workers=8)

2987 segments to load


Loading segments:   3%|▎         | 99/2987 [00:16<03:02, 15.83it/s] 

In [ ]:
for token_dict in loader.tqdm(index, total=len(index)):
    for k in list(token_dict.keys()):
        del token_dict[k]
        token_dict[k] = None
    gc.collect()
del token_dict, k
gc.collect()


  0%|          | 1/14930836 [00:01<4794:44:07,  1.16s/it]


RuntimeError: dictionary keys changed during iteration

In [ ]:
assert len(index) == len(loader.all_tokens)

In [ ]:
table = utils.construct_bool_table(index, all_tokens=loader.all_tokens, valid_books=loader.processed_books)

  0%|          | 0/14930836 [00:00<?, ?it/s]

In [2]:
part_size = 20000
num_books = len(loader.processed_books)
num_parts = num_books // part_size
if num_books % part_size:
    num_parts += 1
i = 0
continue_from = 2

In [3]:
with concurrent.futures.ProcessPoolExecutor(max_workers=8) as pool:
    for i in range(continue_from, num_parts):
        gc.collect()
        print(f"\nPart {i:02d}", flush=True)
        indexing.build_full_index(pool, offset=i*part_size, k=part_size, batch_size=500, index_type="inverted", prefix=f"part{i:02d}")


Part 02


Submitting jobs:   0%|          | 0/20000 [00:00<?, ?it/s]

Initialising...:   0%|          | 0/20000 [00:00<?, ?it/s]

0/20000 failures while building index


Dumping (pickle):   0%|          | 0/2987 [00:00<?, ?it/s]


Part 03


Submitting jobs:   0%|          | 0/10974 [00:00<?, ?it/s]

Initialising...:   0%|          | 0/10974 [00:00<?, ?it/s]

0/10974 failures while building index


Dumping (pickle):   0%|          | 0/2987 [00:00<?, ?it/s]

In [4]:
gc.collect()
indexing.merge_index()

2987 segments to merge
Finished merging parts for 2987 segments...
All done


In [2]:
utils.pickle_to_json()

2987 segments to load


Saving json segments:   0%|          | 0/2987 [00:00<?, ?it/s]

In [6]:
import json
with open("index_json/0000_merged.json") as f:
    index_slice = json.load(f, object_hook=lambda x: {int(k) : v for k, v in x.items()})
x

[{},
 {62053: [1155],
  62066: [1130,
   1131,
   1657,
   1659,
   3370,
   3500,
   3534,
   3676,
   3677,
   3678,
   3679,
   3722,
   5024,
   10678,
   10680,
   10682,
   10688,
   10690,
   10799,
   10805,
   10806,
   10812,
   10929,
   10930,
   10931,
   10932,
   10933,
   10934,
   11047,
   11062,
   11080,
   11085,
   11097,
   11153,
   12116,
   12124,
   12132,
   12140,
   12148,
   12156,
   12164,
   12174,
   12183,
   12224,
   13451,
   13455,
   13461,
   13475,
   13477,
   13503,
   13509,
   13515,
   13525,
   14800,
   15048,
   15373,
   15396,
   15440,
   15462,
   15483,
   15506,
   15571,
   15595,
   15617,
   15639,
   15749,
   15773,
   15775,
   15785,
   15828,
   15833,
   15838,
   17109,
   17114,
   17180,
   17208,
   17210,
   17213,
   17215,
   17247,
   17254,
   17261,
   17266,
   17277,
   17293,
   17318,
   17345,
   17350,
   17356,
   17358,
   17393,
   17401,
   17409,
   17415,
   17423,
   17425,
   17436,
   17437,
   1

In [ ]:
import numpy as np
import scipy.sparse
from pympler.asizeof import asizeof

%load_ext line_profiler

In [3]:
%lprun -f indexing.build_full_index indexing.build_full_index(offset=i*part_size, k=part_size, batch_size=500, index_type="inverted", prefix=f"part{i:02d}")

Submitting jobs:   0%|          | 0/20000 [00:00<?, ?it/s]

Initialising...:   0%|          | 0/20000 [00:00<?, ?it/s]

0/20000 failures while building index


Timer unit: 1e-07 s

Total time: 8073.43 s
File: c:\Users\10022\Documents\GitHub\ttds-group-project\KeywordSearch\indexing.py
Function: build_full_index at line 39

Line #      Hits         Time  Per Hit   % Time  Line Contents
    39                                           def build_full_index(offset: int=0, k: int=-1, batch_size: int=50, index_type: str="inverted", prefix: str="", skip_pickle: bool=False) -> tuple[list, list]:
    40         1          6.0      6.0      0.0      assert index_type in ("bow", "inverted"), "index_type must be \"bow\" or \"inverted\""
    41                                           
    42         2        788.0    394.0      0.0      with open("processed_books.pkl", "rb") as f:
    43         1      21244.0  21244.0      0.0          processed_books = pickle.load(f)
    44         2       1320.0    660.0      0.0      with open("all_tokens.pkl", "rb") as f:
    45         1    9885855.0    1e+07      0.0          _, _, all_tokens = pickle.load(f)
   

In [2]:
loader.process_first_k_books(k=-1, batch_size=50)

70974 books to process


Initialising...:   0%|          | 0/70974 [00:00<?, ?it/s]

0/70974 pre-processing jobs failed:
- 


In [5]:
gc.collect()
index = loader.load_merged_index(save_merged=False)

2987 segments to load


  0%|          | 0/2987 [00:00<?, ?it/s]

  0%|          | 0/2987 [00:00<?, ?it/s]


Garbage collection done
The index took 55 seconds to load
All done


In [ ]:
from pympler.asizeof import asizeof

In [8]:
index[10]

{}

In [4]:
len(loader.all_tokens)

0

In [2]:
and_set = indexing.stemmer.stemWords(["fine", "okay", "happy"])
good = indexing.stemmer.stemWord("good")
not_great = indexing.stemmer.stemWord("great")
query = "fine AND (okay AND (good OR NOT great)) AND happy"
result, _ = kwsearch._bool_search(query, debug=True)
len(result)

['fine AND ', 'okay AND (good OR NOT great)', ' AND', ' happy']
['okay AND ', 'good OR NOT great', None, '']


513

In [3]:
from tqdm import tqdm

num_result = len(result)
for book_id in tqdm(result, total=num_result, desc=f"Validating {num_result} results"):
    with open(loader.token_dir + f"PG{book_id}_tokens.txt", 'r', encoding="UTF-8", errors="ignore") as f:
        tokens = set(indexing.stemmer.stemWords(f.read().splitlines()))
    assert all((term in tokens) for term in and_set) and ((good in tokens) or (not_great not in tokens))

Validating 513 results: 100%|██████████| 513/513 [00:14<00:00, 35.09it/s]


In [4]:
%lprun -f kwsearch.bool_search kwsearch.bool_search(query)

Timer unit: 1e-07 s

Total time: 0.0076213 s
File: c:\Users\10022\Documents\GitHub\ttds-group-project\KeywordSearch\kwsearch.py
Function: bool_search at line 49

Line #      Hits         Time  Per Hit   % Time  Line Contents
    49                                           def bool_search(query: str, debug: bool=False) -> set:
    50         2          6.0      3.0      0.0      if debug:
    51                                                   print(regex_bracket.split(query))
    52         2         68.0     34.0      0.1      tokens = (bool_search_atomic(token, debug) for token in regex_bracket.split(query) if token)
    53         2          8.0      4.0      0.0      is_not = is_and = is_or = False
    54         2      19306.0   9653.0     25.3      valid, (is_not, is_and, is_or) = next(tokens)
    55         6      50494.0   8415.7     66.3      for token_eval, (is_not_, is_and_, is_or_) in tokens:
    56         4         73.0     18.2      0.1          if isinstance(token_eva

In [7]:
phrase_query = "Upon a paper attached to the Narrative".casefold().split(' ')
kwsearch.phrase_search(phrase_query, index)

{10007, 37174}

In [8]:
%lprun -f kwsearch.phrase_search kwsearch.phrase_search(phrase_query, index)

Timer unit: 1e-07 s

Total time: 0.339141 s
File: c:\Users\10022\Documents\GitHub\ttds-group-project\KeywordSearch\kwsearch.py
Function: phrase_search at line 134

Line #      Hits         Time  Per Hit   % Time  Line Contents
   134                                           def phrase_search(words: list[str], index: list[dict] | tuple[dict], debug: bool=False):
   135         1          5.0      5.0      0.0      search_result = []
   136         1          2.0      2.0      0.0      if debug:
   137                                                   print([stemmer.stemWord(word) for word in words if word not in stopwords_set])
   138         1        214.0    214.0      0.0      word_ids = [token_index_dict[stemmer.stemWord(word)] for word in words if word not in stopwords_set]
   139         1         43.0     43.0      0.0      index_entries = [index[i] for i in word_ids]
   140         1         19.0     19.0      0.0      first = list(set(word_ids))
   141         1     170087.0 1

In [9]:
[kwsearch.stemmer.stemWord(word) for word in phrase_query if word not in indexing.stopwords_set]

['upon', 'paper', 'attach', 'narrat']

In [55]:
from KeywordSearch.kwsearch import stemmer, token_index_dict, lookup_table, book_index
from KeywordSearch.indexing import stopwords_set
import numpy as np

def phrase_search(words: list[str], debug: bool=False):
    search_result = []
    if debug:
        print([stemmer.stemWord(word) for word in words if word not in stopwords_set])
    word_ids = [token_index_dict[stemmer.stemWord(word)] for word in words if word not in stopwords_set]
    index_entries = [index[i] for i in word_ids]
    first = list(set(word_ids))
    intersection = lookup_table[first.pop(), :].indices
    for token_id in first:
        intersection = np.intersect1d(intersection, lookup_table[token_id, :].indices, assume_unique=True)
    del first, word_ids

    for docID in intersection:
        occurs = (entry[docID] for entry in index_entries) # use generator to avoid wasting time on non-matches
        first = next(occurs)
        second = next(occurs)
        matches: np.ndarray = second[first[np.searchsorted(first, second, side="right")-1] == second-1]
        del first, second

        if matches.shape[0]:
            for entry in occurs:
                i = np.searchsorted(matches, entry, side="right")
                i[i==0] = 1
                matches = entry[matches[i-1] == entry-1]
                if not matches.any():
                    break
            else:
                search_result.append(docID)
    return set(search_result)

In [56]:
phrase_search("Upon a paper attached to the Narrative".casefold().split(' '))

{10007, 37174}

In [ ]:
gc.collect()
from pympler.asizeof import asizeof
index_size = asizeof(index)
mb = 1024 * 1024
gb = mb * 1024
gb_size = index_size // gb
mb_size = round((index_size % gb) / mb)
print(f"{gb_size}GB" + f" {mb_size}MB" if mb_size else '')

44GB 370MB


In [ ]:
all_tokens = loader.load_token_vocab(k=-1)

In [ ]:
import pickle
with open("valid_books.pkl", "rb") as f:
    _, _, valid_books = pickle.load(f)

In [ ]:
len(all_tokens), len(index)

(5696023, 5696023)

In [ ]:
from typing import Iterable

import numpy as np
import scipy.sparse
try:
    from tqdm.notebook import tqdm
    USE_TQDM = True
except:
    USE_TQDM = False

def construct_bool_table(index: Iterable[dict], save_path: str|None=None):
    table = scipy.sparse.dok_matrix((len(all_tokens), max(valid_books) + 1), dtype=np.bool_)
    print(len(index), table.shape)
    length = len(all_tokens)
    tqdm_iter = enumerate(index[:length])
    if USE_TQDM:
        tqdm_iter = tqdm(tqdm_iter, total=length)
    for token_id, token_dict in tqdm_iter:
        if token_dict:
            table[token_id, tuple(token_dict.keys())] = True
    gc.collect()
    table = table.tocsr()
    if save_path is None:
        return table
    else:
        gc.collect()
        scipy.sparse.save_npz("lookup_table.npz", table, compressed=True)
        return table

In [ ]:
(table.data.nbytes + table.indptr.nbytes + table.indices.nbytes) // 1024 // 1024

1063

In [ ]:
dok_table: scipy.sparse.dok_matrix = table.todok()

In [ ]:
asizeof(dok_table) // 1024 // 1024

In [ ]:
asizeof(table) // 1024 // 1024

In [ ]:
scipy.sparse.save_npz("lookup_table_coo.npz", table.tocoo(), compressed=True)

In [ ]:
x = list(range(5696023))
y = []
index_size = len(x)
batch_size = 5000
batches = index_size // batch_size
for i in range(batches):
    start = i * batch_size
    end = min(start + batch_size, index_size)
    y.extend(x[start:end])

y.extend(x[end:])
x == y, end

(True, 5695000)

In [ ]:
with open("all_tokens_old.pkl", 'rb') as f:
    _, _, all_tokens_old = pickle.load(f)
len(all_tokens_old)

5696023

In [ ]:
# inverted_index_2, indexed_books_2 = indexing.build_full_index(offset=5000, k=5000, batch_size=-1, index_type="inverted", prefix="part02")
# import h5py
# def save_inv_index_HDF5(filename: str, index: list[dict], **kwargs):
#     with h5py.File(filename, 'w') as f:
#         for i, entry in enumerate(index):
#             group = f.create_group(str(i))
#             for book_id, occurrences in entry.items():
#                 group.create_dataset(str(book_id), data=occurrences, **kwargs)
# save_inv_index_HDF5("test.h5", inverted_index_1, chunks=True, compression="gzip")

In [ ]:
import pickle
import numpy as np

with open("index/inverted_113.pkl", "rb") as f:
    first = pickle.load(f)

In [ ]:
len(first)


In [ ]:
#bow_index, indexed_books = indexing.build_full_index(k=-1)

In [ ]:
import pickle
import numpy as np

with open("index.pkl", "rb") as f:
    bow_index = pickle.load(f)
# dummy_arr = np.array([], dtype=np.uint8)
# dummy = (0, dummy_arr, 0, dummy_arr)
# bow_index = tuple([dummy] + [pair if isinstance(pair, tuple) else dummy for pair in bow_index[1:]])
# with open("index.pkl", "wb") as f:
#     pickle.dump(bow_index, f)

with open("valid_books.pkl", "rb") as f:
    _, _, valid_books = pickle.load(f)

book_list = sorted(valid_books)

non_empty_books = []
non_empty_index = []
for book_id, book_bow in enumerate(bow_index):
    if book_bow:
        non_empty_books.append(book_id)
        non_empty_index.append(book_bow)

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity
import concurrent.futures

In [ ]:
# corpus = [utils.deltazip([vocab, counts], [vocab_delta, count_delta]) for vocab_delta, vocab, count_delta, counts in non_empty_index]
# tfidf_model = TfidfModel(corpus, smartirs='ntc')
# with open("tfidf_model.pkl", "wb") as f:
#     pickle.dump(tfidf_model, f)
with open("tfidf_model.pkl", "rb") as f:
    tfidf_model = pickle.load(f)

In [ ]:
dim = len(all_tokens) + 1
def cast2numpy(x):
    arr = np.zeros(dim, dtype=np.float16)
    for i, score in x:
        arr[i] = score
    return arr

In [ ]:
first = [tfidf_model[utils.deltazip([vocab, counts], [vocab_delta, count_delta])] for vocab_delta, vocab, count_delta, counts in non_empty_index]
# with open("tfidf_index.pkl", "rb") as f:
#     tmp = pickle.load(f)

In [ ]:
tfidf_model[[(0,1),(3,8)]]

In [ ]:
vocab_delta, vocab, count_delta, counts = non_empty_index[0]
cast2numpy(tfidf_model[utils.deltazip([vocab, counts], [vocab_delta, count_delta])]).mean()

In [ ]:
dim = len(all_tokens) + 1

In [ ]:
m1 = SparseMatrixSimilarity(first, num_best=10, num_features=dim)

In [ ]:
with open("tfidf_index.pkl", "wb") as f:
    pickle.dump(first, f)
with open("matrix.pkl", "wb") as f:
    pickle.dump(m1, f)

In [ ]:
asizeof.asizeof(first)/1024/1024/1024

In [ ]:
asizeof.asizeof(m1)/1024/1024/1024

In [ ]:
vocab_delta, vocab, count_delta, counts = non_empty_index[269]
next(utils.deltazip([vocab, counts], [vocab_delta, count_delta]))

In [ ]:
non_empty_index[269]

In [ ]:
tokens = [1, 3, 1, 9, 3, 5654]
vocab_list = sorted(set(tokens))
token_arr = np.array(tokens)
vocab, vocab_delta = utils.cast2intarr(vocab_list)
counts, count_delta = utils.cast2intarr([np.sum(token_arr == token) for token in vocab_list])
(vocab_delta, vocab, count_delta, counts, vocab_list, [(vocab == token-vocab_delta) for token in vocab_list])
vocab + vocab_delta, counts + count_delta

In [ ]:
book_path_template = loader.token_dir + "PG%d_tokens.txt"

def read_tokens(PG_id: int):
    with open(book_path_template %PG_id, encoding="UTF-8", errors="ignore") as f:
        return loader.stemmer.stemWords(f.read().splitlines())

In [ ]:


docs = []
docs_index = []
failed_jobs = 0
complete_counter = 0
with concurrent.futures.ThreadPoolExecutor() as pool:
    jobs = {
        pool.submit(
            read_tokens, book_id)
            : book_id for book_id in indexed_books
        }
    
    for job in concurrent.futures.as_completed(jobs):
        book_id = jobs[job]
        try:
            result = job.result()
            docs.append(result)
            docs_index.append(book_id)
        except Exception as e:
            # raise e
            failed_jobs.append(book_id)
        complete_counter += 1
        print(f"Finished fetching tokens in {complete_counter} books...", end="\r")
        
        print(f"\n{len(failed_jobs)}/{len(jobs)} token fetching jobs failed")

In [ ]:
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]
del docs
tfidf = TfidfModel(corpus, smartirs='ntc')
tfidf_corpus = [tfidf[doc] for doc in corpus]
del corpus, tfidf
index = MatrixSimilarity(tfidf_corpus, num_features=len(dictionary))

In [ ]:
# import pickle
# with open("all_tokens.pkl", 'rb') as f:
#     k, offset, _all_tokens = pickle.load(f)
# with open("all_tokens.pkl", "wb") as f:
#     pickle.dump((k, offset, tuple(_all_tokens)), f)

In [ ]:
from pympler import asizeof
asizeof.asizeof(bow_index), asizeof.asizeof(index)

In [ ]:
415963552/1024/1024

In [ ]:
415971552/1024/1024

In [ ]:

mem_size_lst = asizeof.asizeof(all_tokens)
mem_size_tup = asizeof.asizeof(tuple(all_tokens))
mem_size_lst/1024/1024, mem_size_tup

In [ ]:
from math import log2
import numpy as np
log2(len(all_tokens))
x = np.array([1,2,3])
x.shape[0]

In [ ]:
#loader.build_full_index()

In [ ]:
import pickle

In [ ]:
with open("index.pkl", "rb") as f:
    index = pickle.load(f)

In [ ]:
from pympler import asizeof
mem_size = asizeof.asizeof(index)

In [ ]:
mem_size/1024/1024

In [ ]:
index[1]

In [ ]:
size = 0
print(len(index))
for i, token_eval in enumerate(index):
    for arr in token_eval.values():
        size += asizeof.asizeof(arr)
    print(i, end='\r')
size

In [ ]:
size/1024/1024

In [16]:
import re
import random

random.seed(2050808)
start, end = ord('a'), ord('z') + 1
r1, r2, r3, r4 = re.compile("\[.*?\]"), re.compile("\[.*\]"), re.compile(r"\b\w+\b"), re.compile(r"\W+")

tmp = [chr(random.randrange(start, end)) for _ in [0] * 1000000]
rand_pos = []
for _ in [0] * 100:
    x = random.randrange(0, 1000000)
    while x in rand_pos:
        x = random.randrange(0, 1000000)
    rand_pos.append(x)
rand_pos.sort()

while rand_pos:
    tmp[rand_pos.pop()] = ']'
    tmp[rand_pos.pop()] = '['

test_str = ''.join(tmp)
test_str2 = ''.join(''.join(chr(random.randrange(start, end)) for _ in [0] * random.randrange(5, 30)) + ' ' * random.randrange(1, 4) for _ in [0] * 100000)

del tmp, rand_pos

In [17]:
%timeit r1.findall(test_str)

2.27 ms ± 60.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit r2.findall(test_str)

260 µs ± 5.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
%timeit r1.findall(test_str)

2.26 ms ± 55 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
%timeit r3.findall(test_str2)

21.2 ms ± 305 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%timeit r4.split(test_str2)

29.4 ms ± 754 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%tim

In [29]:
import nltk
nltk.tokenize.sent_tokenize

<function nltk.tokenize.sent_tokenize(text, language='english')>